<a href="https://colab.research.google.com/github/ljs7463/AnalysisProject/blob/master/%EB%8D%B0%EC%9D%B4%EC%BD%98/analytics/xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

# 시각화 폰트 설정
if os.name =='posix':
    plt.rc("font", family = "AppleGothic")

else:
    plt.rc("font", family = "Malgun Gothic")

# 경고문자 무시
warnings.filterwarnings(action='ignore')

In [2]:
# 코랩 실행시
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_sub = pd.read_csv('sample_submission.csv')
df_info = pd.read_csv('data_info.csv')

In [3]:
## Label Encoding

## train데이터
df_train['preferred_difficulty_level'] = pd.factorize(df_train['preferred_difficulty_level'])[0]
df_train['subscription_type'] = pd.factorize(df_train['subscription_type'])[0]

## test데이터
df_test['preferred_difficulty_level'] = pd.factorize(df_test['preferred_difficulty_level'])[0]
df_test['subscription_type'] = pd.factorize(df_test['subscription_type'])[0]

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
for i in [['subscription_duration','recent_login_time','average_time_per_learning_session','monthly_active_learning_days','total_completed_courses','recent_learning_achievement','abandoned_learning_sessions','community_engagement_level','customer_inquiry_history','payment_pattern']]:
  df_train[i] = scaler.fit_transform(df_train[i])
for i in [['subscription_duration','recent_login_time','average_time_per_learning_session','monthly_active_learning_days','total_completed_courses','recent_learning_achievement','abandoned_learning_sessions','community_engagement_level','customer_inquiry_history','payment_pattern']]:
  df_test[i] = scaler.transform(df_test[i])

In [5]:
# Delete user_id
df_train = df_train.drop(columns = 'user_id')

# split target
x = df_train[list(df_train.columns[:-1])]
y = df_train['target']

In [6]:
# Delete user_id
df_test = df_test.drop(columns = 'user_id')

# split target
new_x = df_test


# xgboost + optuna

In [7]:
import optuna
from sklearn.model_selection import KFold
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import make_scorer, f1_score
from sklearn.datasets import load_iris
import pandas as pd

def objective(trial, x, y):
  # Optuna가 탐색할 하이퍼파라미터의 범위 설정
  params = {
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        'n_estimators': trial.suggest_int('n_estimators',1000,1001),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'eval_metric': 'logloss',
        'use_label_encoder': False
    }

  # k-겹 교차 검증 설정
  kf = KFold(
      n_splits =10,
      shuffle = True,
      random_state = 42)
  f1_scores = []

  for train_index, test_index in kf.split(x):
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # xgBoost 모델생성
    model = XGBClassifier(**params, n_jobs = -1)
    model.fit(x_train, y_train, eval_set = [(x_test, y_test)], early_stopping_rounds = 100)

    # 예측 및 F 점수 계산
    preds = model.predict(x_test)
    f1 = f1_score(y_test, preds, average = 'macro')
    f1_scores.append(f1)

  return np.mean(f1_scores)

# Optuna  스터디 생성 및 최적화 실행
study = optuna.create_study(direction = 'maximize')
# lambda 함수를 사용하여 x와 y를 objective 함수에 전달
study.optimize(lambda trial: objective(trial, x, y), n_trials = 40)

# 최적 하이퍼파라미터 출력
print('Best trial :', study.best_trial.params)

[I 2023-12-03 14:06:44,075] A new study created in memory with name: no-name-038e2982-9184-4f2e-a0bd-ef227b774652


[0]	validation_0-logloss:0.66270
[1]	validation_0-logloss:0.66274
[2]	validation_0-logloss:0.66282
[3]	validation_0-logloss:0.66282
[4]	validation_0-logloss:0.66278
[5]	validation_0-logloss:0.66295
[6]	validation_0-logloss:0.66288
[7]	validation_0-logloss:0.66289
[8]	validation_0-logloss:0.66280
[9]	validation_0-logloss:0.66261
[10]	validation_0-logloss:0.66250
[11]	validation_0-logloss:0.65947
[12]	validation_0-logloss:0.65952
[13]	validation_0-logloss:0.65972
[14]	validation_0-logloss:0.65958
[15]	validation_0-logloss:0.65940
[16]	validation_0-logloss:0.65948
[17]	validation_0-logloss:0.65940
[18]	validation_0-logloss:0.65957
[19]	validation_0-logloss:0.65954
[20]	validation_0-logloss:0.65987
[21]	validation_0-logloss:0.65985
[22]	validation_0-logloss:0.65992
[23]	validation_0-logloss:0.65742
[24]	validation_0-logloss:0.65743
[25]	validation_0-logloss:0.65717
[26]	validation_0-logloss:0.65710
[27]	validation_0-logloss:0.65707
[28]	validation_0-logloss:0.65694
[29]	validation_0-loglos

[I 2023-12-03 14:07:03,579] Trial 0 finished with value: 0.38652456842822025 and parameters: {'max_depth': 3, 'learning_rate': 0.08912511293397372, 'n_estimators': 1001, 'min_child_weight': 8, 'gamma': 0.002413228804903361, 'subsample': 0.5469146404666146, 'colsample_bytree': 0.0520812309854585, 'reg_alpha': 6.951795703539565e-06, 'reg_lambda': 4.556304505874364e-05}. Best is trial 0 with value: 0.38652456842822025.


[0]	validation_0-logloss:0.66277
[1]	validation_0-logloss:0.66284
[2]	validation_0-logloss:0.66290
[3]	validation_0-logloss:0.66290
[4]	validation_0-logloss:0.66303
[5]	validation_0-logloss:0.66301
[6]	validation_0-logloss:0.66299
[7]	validation_0-logloss:0.66298
[8]	validation_0-logloss:0.66294
[9]	validation_0-logloss:0.66289
[10]	validation_0-logloss:0.66300
[11]	validation_0-logloss:0.66267
[12]	validation_0-logloss:0.66265
[13]	validation_0-logloss:0.66249
[14]	validation_0-logloss:0.66258
[15]	validation_0-logloss:0.66245
[16]	validation_0-logloss:0.66243
[17]	validation_0-logloss:0.66245
[18]	validation_0-logloss:0.66237
[19]	validation_0-logloss:0.66232
[20]	validation_0-logloss:0.66226
[21]	validation_0-logloss:0.66221
[22]	validation_0-logloss:0.66221
[23]	validation_0-logloss:0.66185
[24]	validation_0-logloss:0.66180
[25]	validation_0-logloss:0.66175
[26]	validation_0-logloss:0.66172
[27]	validation_0-logloss:0.66173
[28]	validation_0-logloss:0.66162
[29]	validation_0-loglos

[I 2023-12-03 14:07:41,085] Trial 1 finished with value: 0.3839937317047218 and parameters: {'max_depth': 5, 'learning_rate': 0.014600778111151435, 'n_estimators': 1000, 'min_child_weight': 5, 'gamma': 0.03081976777315682, 'subsample': 0.028795567851560865, 'colsample_bytree': 0.13801600135123523, 'reg_alpha': 1.1670561291471645e-07, 'reg_lambda': 6.591359997752397e-05}. Best is trial 0 with value: 0.38652456842822025.


...

[0]	validation_0-logloss:0.66325
[1]	validation_0-logloss:0.66371
[2]	validation_0-logloss:0.66385
[3]	validation_0-logloss:0.66424
[4]	validation_0-logloss:0.66476
[5]	validation_0-logloss:0.66469
[6]	validation_0-logloss:0.66457
[7]	validation_0-logloss:0.66456
[8]	validation_0-logloss:0.66444
[9]	validation_0-logloss:0.66399
[10]	validation_0-logloss:0.66437
[11]	validation_0-logloss:0.66311
[12]	validation_0-logloss:0.66297
[13]	validation_0-logloss:0.66239
[14]	validation_0-logloss:0.66308
[15]	validation_0-logloss:0.66314
[16]	validation_0-logloss:0.66326
[17]	validation_0-logloss:0.66282
[18]	validation_0-logloss:0.66243
[19]	validation_0-logloss:0.66238
[20]	validation_0-logloss:0.66218
[21]	validation_0-logloss:0.66205
[22]	validation_0-logloss:0.66198
[23]	validation_0-logloss:0.66065
[24]	validation_0-logloss:0.66004
[25]	validation_0-logloss:0.65991
[26]	validation_0-logloss:0.65974
[27]	validation_0-logloss:0.65976
[28]	validation_0-logloss:0.65933
[29]	validation_0-loglos

[I 2023-12-03 14:37:38,565] Trial 38 finished with value: 0.393399124172771 and parameters: {'max_depth': 5, 'learning_rate': 0.04840572762729458, 'n_estimators': 1000, 'min_child_weight': 3, 'gamma': 1.9214455892867856e-05, 'subsample': 0.028012186187776043, 'colsample_bytree': 0.09538978962150851, 'reg_alpha': 0.00637852616331669, 'reg_lambda': 5.525254061438429e-08}. Best is trial 19 with value: 0.4350282151579668.


[0]	validation_0-logloss:0.66381
[1]	validation_0-logloss:0.66404
[2]	validation_0-logloss:0.66431
[3]	validation_0-logloss:0.66436
[4]	validation_0-logloss:0.66477
[5]	validation_0-logloss:0.66515
[6]	validation_0-logloss:0.66503
[7]	validation_0-logloss:0.66498
[8]	validation_0-logloss:0.66486
[9]	validation_0-logloss:0.66455
[10]	validation_0-logloss:0.66547
[11]	validation_0-logloss:0.66355
[12]	validation_0-logloss:0.66337
[13]	validation_0-logloss:0.66280
[14]	validation_0-logloss:0.66363
[15]	validation_0-logloss:0.66350
[16]	validation_0-logloss:0.66314
[17]	validation_0-logloss:0.66412
[18]	validation_0-logloss:0.66455
[19]	validation_0-logloss:0.66408
[20]	validation_0-logloss:0.66407
[21]	validation_0-logloss:0.66472
[22]	validation_0-logloss:0.66431
[23]	validation_0-logloss:0.66367
[24]	validation_0-logloss:0.66354
[25]	validation_0-logloss:0.66298
[26]	validation_0-logloss:0.66257
[27]	validation_0-logloss:0.66242
[28]	validation_0-logloss:0.66197
[29]	validation_0-loglos

[I 2023-12-03 14:38:28,522] Trial 39 finished with value: 0.41372475414748566 and parameters: {'max_depth': 5, 'learning_rate': 0.06295618871927465, 'n_estimators': 1000, 'min_child_weight': 2, 'gamma': 3.668202076758783e-06, 'subsample': 0.014491635548229444, 'colsample_bytree': 0.13027921924110106, 'reg_alpha': 0.0006748673023641385, 'reg_lambda': 1.3496199135259854e-07}. Best is trial 19 with value: 0.4350282151579668.


Best trial : {'max_depth': 8, 'learning_rate': 0.05751034807598742, 'n_estimators': 1000, 'min_child_weight': 1, 'gamma': 8.65827489338555e-05, 'subsample': 0.04996949287760562, 'colsample_bytree': 0.19045686677444706, 'reg_alpha': 0.00028382833243660236, 'reg_lambda': 1.215024724833776e-08}


In [9]:
# Optuna 스터디에서 최적의 하이퍼파라미터 가져오기
best_params = study.best_trial.params

# 최적의 하이퍼파라미터를 사용하여 CatBoost 모델 초기화
final_model = XGBClassifier(**best_params, verbose=0)

# 전체 훈련 데이터셋을 사용하여 최종 모델 훈련
final_model.fit(x, y)

# 테스트 데이터셋에 대한 예측 수행
predictions = final_model.predict(new_x)

# 여기서 X_train, y_train, X_test는 실제 데이터셋을 가리킵니다.
# X_train, y_train은 모델 훈련에 사용되는 전체 훈련 데이터셋이고,
# X_test는 최종 예측을 수행할 테스트 데이터셋입니다.

In [10]:
# 예측값 저장
df_sub['target'] = predictions
df_sub.set_index('user_id').to_csv('xgboost_optuna(basic).csv', encoding="cp949")

In [ ]:
afrom xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import make_scorer, f1_score
from sklearn.datasets import load_iris
import pandas as pd

# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y,random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train,random_state=42)
# param_grid = {
#     'max_depth': [3,4,5,6],
#     'learning_rate': [0.1, 0.01, 0.05],
#     'n_estimators': [100,200,300,400,500, 600],
#     'colsample_bytree': [0.3, 0.7, 0.9]
# }

# XGBoost 분류기 초기화
xgb = XGBClassifier()

# parameter grid
xgb_param_grid={
    'objective':['binary:logistic'],
    'n_estimators' : [600,700],
    'learning_rate' : [0.01,0.05,0.1],
    'max_depth' : [3,5,7,10],
    'gamma' : [0,1,2,3],
    'colsample_bytree' : [0.3,0.6,0.9],

}


# GridSearchCV 설정
grid_search = GridSearchCV( xgb ,
                            param_grid =xgb_param_grid,
                            cv=5,
                            scoring = "f1_macro",
                            n_jobs=-1,
                            verbose=1
                            )

# GridSearchCV 실행
grid_search.fit(x_train, y_train,early_stopping_rounds = 100,eval_metric='logloss', eval_set = [(x_val,y_val)])

# 결과 출력
print("최적의 파라미터:", grid_search.best_params_)
print("최고 평균 정확도:", grid_search.best_score_)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits
[0]	validation_0-logloss:0.66375
[1]	validation_0-logloss:0.66128
[2]	validation_0-logloss:0.65999
[3]	validation_0-logloss:0.65817
[4]	validation_0-logloss:0.65859
[5]	validation_0-logloss:0.65847
[6]	validation_0-logloss:0.65856
[7]	validation_0-logloss:0.65890
[8]	validation_0-logloss:0.65889
[9]	validation_0-logloss:0.65870
[10]	validation_0-logloss:0.65773
[11]	validation_0-logloss:0.65615
[12]	validation_0-logloss:0.65613
[13]	validation_0-logloss:0.65615
[14]	validation_0-logloss:0.65674
[15]	validation_0-logloss:0.65685
[16]	validation_0-logloss:0.65644
[17]	validation_0-logloss:0.65583
[18]	validation_0-logloss:0.65588
[19]	validation_0-logloss:0.65585
[20]	validation_0-logloss:0.65575
[21]	validation_0-logloss:0.65503
[22]	validation_0-logloss:0.65513
[23]	validation_0-logloss:0.65450
[24]	validation_0-logloss:0.65403
[25]	validation_0-logloss:0.65386
[26]	validation_0-logloss:0.65372
[27]	validation_0-logloss:0.

In [ ]:
grid_search.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.3, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=600, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
# Save the best Model
best_model = grid_search.best_estimator_

# predictions
y_pred = best_model.predict(x_test)

# Estimate Macro F1 Scores
macro_f1 = f1_score(y_test, y_pred, average = 'macro')
print(f"Test SEt Macro F1 Score: {macro_f1}")

Test SEt Macro F1 Score: 0.4281248500905217


In [ ]:
predictions = best_model.predict(new_x)

In [ ]:
df_sub['target'] = predictions
df_sub.set_index('user_id').to_csv('xgboost.csv',encoding = 'cp949')